In [1]:
pip install roboflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 751.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.4 MB/s eta 0:00:00
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
      Successfully uninstalled chardet-5.2.0


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
import requests
import json
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set your Roboflow API key and other parameters
api_key = "ufSklXR1QLMZRFvcBad7"
model_id = "ship-detection-dg5bz"
image_folder_path = "/content/drive/My Drive/images" # Modified path to include 'My Drive'
output_folder_path = "/content/drive/My Drive/predictions" # Modified path to include 'My Drive'

# Function to get predictions for an image
def get_predictions(image_path):
    url = f"https://detect.roboflow.com/ship-detection-dg5bz/1"
    params = {
        "api_key": api_key
    }
    with open(image_path, "rb") as image_file:
        response = requests.post(url, params=params, files={"file": image_file})
        response.raise_for_status()
        return response.json()

# Function to save predictions to a JSON file
def save_predictions(predictions, output_path):
    with open(output_path, "w") as output_file:
        json.dump(predictions, output_file, indent=4)

# Loop through images and get/save predictions
for image_name in os.listdir(image_folder_path):
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder_path, image_name)
        predictions = get_predictions(image_path)

        output_path = os.path.join(output_folder_path, f"{os.path.splitext(image_name)[0]}_predictions.json")
        save_predictions(predictions, output_path)
        print(f"Saved predictions for {image_name} to {output_path}")

print("Finished processing all images.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Saved predictions for Test97_jpg.rf.142d460fb95d0ff36f204433e0af6be9.jpg to /content/drive/My Drive/predictions/Test97_jpg.rf.142d460fb95d0ff36f204433e0af6be9_predictions.json
Saved predictions for Test19_jpg.rf.3893297965a970217d27c02499033bf3.jpg to /content/drive/My Drive/predictions/Test19_jpg.rf.3893297965a970217d27c02499033bf3_predictions.json
Saved predictions for GE_197_jpg.rf.73898ccebcda5f2c6508425055b837e2.jpg to /content/drive/My Drive/predictions/GE_197_jpg.rf.73898ccebcda5f2c6508425055b837e2_predictions.json
Saved predictions for GE_340_jpg.rf.e555058d6b248eb83b030f48b717beb5.jpg to /content/drive/My Drive/predictions/GE_340_jpg.rf.e555058d6b248eb83b030f48b717beb5_predictions.json
Saved predictions for Test5_jpg.rf.50d5889712d23dd8fefc5c1e68d50ac7.jpg to /content/drive/My Drive/predictions/Test5_jpg.rf.50d5889712d23dd8fefc5c1e68d50ac7_prediction

In [21]:
import requests
import json
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set your Roboflow API key and other parameters
api_key = "ufSklXR1QLMZRFvcBad7"
model_id = "ship-detection-dg5bz"
image_folder_path = "/content/drive/My Drive/images"
output_folder_path = "/content/drive/My Drive/predictions"

# Ensure the output folder exists
os.makedirs(output_folder_path, exist_ok=True)

# Function to get predictions for an image
def get_predictions(image_path):
    url = f"https://detect.roboflow.com/{model_id}/1"
    params = {
        "api_key": api_key
    }
    with open(image_path, "rb") as image_file:
        response = requests.post(url, params=params, files={"file": image_file})
        response.raise_for_status()
        return response.json()

# Function to save predictions to a JSON file
def save_predictions(predictions, output_path):
    with open(output_path, "w") as output_file:
        json.dump(predictions, output_file, indent=4)

# Function to calculate surface area and categorize ships
def categorize_ships(predictions):
    categorized_predictions = {
        "image": predictions["image"],
        "predictions": []
    }
    categories = {'small': [], 'medium': [], 'large': []}

    for pred in predictions['predictions']:
        width = pred['width']
        height = pred['height']
        surface_area = width * height
        pred['surface_area'] = surface_area

        # Categorize based on surface area
        if surface_area < 1000:
            pred['category'] = 'small'
            categories['small'].append(pred)
        elif surface_area < 5000:
            pred['category'] = 'medium'
            categories['medium'].append(pred)
        else:
            pred['category'] = 'large'
            categories['large'].append(pred)

        categorized_predictions["predictions"].append(pred)

    return categories, categorized_predictions

# Loop through images and get/save predictions
for image_name in os.listdir(image_folder_path):
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder_path, image_name)
        try:
            predictions = get_predictions(image_path)
            output_path = os.path.join(output_folder_path, f"{os.path.splitext(image_name)[0]}_predictions.json")
            save_predictions(predictions, output_path)

            # Categorize ships based on surface area
            categories, categorized_predictions = categorize_ships(predictions)

            # Save categorized predictions to a new JSON file
            categorized_output_path = os.path.join(output_folder_path, f"{os.path.splitext(image_name)[0]}_categorized_predictions.json")
            save_predictions(categorized_predictions, categorized_output_path)

            print(f"Categories for {image_name}: {categories}")
        except requests.exceptions.HTTPError as err:
            print(f"Failed to get predictions for {image_name}: {err}")

print("Finished processing all images.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Categories for Test97_jpg.rf.142d460fb95d0ff36f204433e0af6be9.jpg: {'small': [], 'medium': [], 'large': []}
Categories for Test19_jpg.rf.3893297965a970217d27c02499033bf3.jpg: {'small': [], 'medium': [], 'large': []}
Categories for GE_197_jpg.rf.73898ccebcda5f2c6508425055b837e2.jpg: {'small': [], 'medium': [], 'large': []}
Categories for GE_340_jpg.rf.e555058d6b248eb83b030f48b717beb5.jpg: {'small': [], 'medium': [], 'large': [{'x': 502.5, 'y': 602.5, 'width': 163.0, 'height': 75.0, 'confidence': 0.858858585357666, 'class': 'ship', 'class_id': 0, 'detection_id': '84263b13-4e44-47fd-bacf-e061e646daf5', 'surface_area': 12225.0, 'category': 'large'}]}
Categories for Test5_jpg.rf.50d5889712d23dd8fefc5c1e68d50ac7.jpg: {'small': [], 'medium': [], 'large': [{'x': 138.0, 'y': 515.0, 'width': 138.0, 'height': 62.0, 'confidence': 0.9172180891036987, 'class': 'ship', 'cla

In [22]:
import requests
import json
import os
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Set your Roboflow API key and other parameters
api_key = "ufSklXR1QLMZRFvcBad7"
model_id = "ship-detection-dg5bz"
image_folder_path = "/content/drive/My Drive/images"

# Function to get predictions for an image
def get_predictions(image_path):
    url = f"https://detect.roboflow.com/{model_id}/1"
    params = {
        "api_key": api_key
    }
    with open(image_path, "rb") as image_file:
        response = requests.post(url, params=params, files={"file": image_file})
        response.raise_for_status()
        return response.json()

# Function to categorize ships and count each category
def categorize_ships(predictions):
    counts = {'small': 0, 'medium': 0, 'large': 0}

    for pred in predictions['predictions']:
        width = pred['width']
        height = pred['height']
        surface_area = width * height

        # Categorize based on surface area
        if surface_area < 1000:
            counts['small'] += 1
        elif surface_area < 5000:
            counts['medium'] += 1
        else:
            counts['large'] += 1

    return counts

# Loop through images and get counts
overall_counts = {'small': 0, 'medium': 0, 'large': 0}
for image_name in os.listdir(image_folder_path):
    if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder_path, image_name)
        try:
            predictions = get_predictions(image_path)

            # Get ship category counts
            counts = categorize_ships(predictions)

            # Update overall counts
            overall_counts['small'] += counts['small']
            overall_counts['medium'] += counts['medium']
            overall_counts['large'] += counts['large']

            print(f"Counts for {image_name}: {counts}")
        except requests.exceptions.HTTPError as err:
            print(f"Failed to get predictions for {image_name}: {err}")

print("Overall counts for all images:")
print(f"Small: {overall_counts['small']}")
print(f"Medium: {overall_counts['medium']}")
print(f"Large: {overall_counts['large']}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Counts for Test97_jpg.rf.142d460fb95d0ff36f204433e0af6be9.jpg: {'small': 0, 'medium': 0, 'large': 0}
Counts for Test19_jpg.rf.3893297965a970217d27c02499033bf3.jpg: {'small': 0, 'medium': 0, 'large': 0}
Counts for GE_197_jpg.rf.73898ccebcda5f2c6508425055b837e2.jpg: {'small': 0, 'medium': 0, 'large': 0}
Counts for GE_340_jpg.rf.e555058d6b248eb83b030f48b717beb5.jpg: {'small': 0, 'medium': 0, 'large': 1}
Counts for Test5_jpg.rf.50d5889712d23dd8fefc5c1e68d50ac7.jpg: {'small': 0, 'medium': 0, 'large': 1}
Counts for Test30_jpg.rf.ef82f1b8be52677dbc48f9e1bcbb2349.jpg: {'small': 0, 'medium': 0, 'large': 0}
Counts for GE_114_jpg.rf.52340ea62489ef99bec116f3eab78582.jpg: {'small': 0, 'medium': 0, 'large': 2}
Counts for Test77_jpg.rf.63dd80c278e8b24a3595b6aaa9c59797.jpg: {'small': 0, 'medium': 0, 'large': 0}
Counts for Test52_jpg.rf.518601f7d26a97405205f95a5447d7ec.jpg: {